# Models

> models


In [ ]:
#| default_exp models

In [ ]:
#| hide
from nbdev.showdoc import *


In [ ]:
#| hide
from IPython.display import clear_output, DisplayHandle

def update_patch(self, obj):
    clear_output(wait=True)
    self.display(obj)
DisplayHandle.update = update_patch

In [ ]:
#| export 

from fastai.vision.all import ConvLayer, Lambda, MaxPool, NormType, nn, np
from torch import cat as torch_cat
from Noise2Model.utils import attributesFromDict

/home/bm/miniconda3/envs/n2m/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from torch import randn as torch_randn
from fastai.vision.all import test_eq

In [ ]:
x = torch_randn(16,1,32,64,64)
xdim = len(x.shape)-2

tst = ConvLayer(1,1, ndim=xdim)
test_eq(tst(x).shape, [16,1,32,64,64])
tst = MaxPool(2,ndim=xdim)
test_eq(tst(x).shape, [16,1,16,32,32])
tst = Lambda(lambda x: x+np.float32(1e-3))
test_eq(tst(x).shape, [16,1,32,64,64])
test_eq(torch_cat((x,tst(x)),1).shape, [16,2,32,64,64])

In [ ]:
#| export

class DnCNN(nn.Module):
    def __init__(self, channels, num_of_layers=9, features=64, kernel_size=3):
        super(DnCNN, self).__init__()
        padding = 1
        layers = []
        layers.append(ConvLayer(channels, features, ks=kernel_size, padding=padding, norm_type=None))
        for _ in range(num_of_layers-2):
            layers.append(ConvLayer(features, features, ks=kernel_size, padding=padding))
        layers.append(nn.Conv2d(in_channels=features, out_channels=channels, kernel_size=kernel_size, padding=padding, bias=False))
        self.dncnn = nn.Sequential(*layers)
    def forward(self, x):
        residual = self.dncnn(x)
        denoised = x - residual
        return denoised

In [ ]:
x = torch_randn(16,1,32,64)

tst = DnCNN(1)
test_eq(tst(x).shape, [16,1,32,64])
print(tst)

DnCNN(
  (dncnn): Sequential(
    (0): ConvLayer(
      (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
    )
    (1): ConvLayer(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (2): ConvLayer(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (3): ConvLayer(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (4): ConvLayer(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, trac

In [ ]:
#| export
def SubNetConv(ks=3, 
            stride=1,
            padding=None,
            bias=None, 
            ndim=2,
            norm_type=NormType.Batch, 
            bn_1st=True, 
            act_cls=nn.ReLU, 
            transpose=False,
            init='auto', 
            xtra=None, 
            bias_std=0.01,
            dropout=0.0,
            ):

    def _conv(n_in ,n_out, n_conv=1):
        s = ConvLayer(n_in,n_out,ks=ks, stride=stride, padding=padding, bias=bias, ndim=ndim, norm_type=norm_type, bn_1st=bn_1st,
                 act_cls=act_cls, transpose=transpose, init=init, xtra=xtra, bias_std=bias_std)
        if dropout is not None and dropout > 0: s = nn.Sequential(s,nn.Dropout(dropout))
        for i in range(n_conv-1):
            t = ConvLayer(n_out,n_out,ks=ks, stride=stride, padding=padding, bias=bias, ndim=ndim, norm_type=norm_type, bn_1st=bn_1st,
                 act_cls=act_cls, transpose=transpose, init=init, xtra=xtra, bias_std=bias_std)
            if dropout is not None and dropout > 0: t = nn.Sequential(t,nn.Dropout(dropout))
            s = nn.Sequential(s,t)
        return s

    return _conv

In [ ]:
x = torch_randn(16,1,32,64,64)
xdim = len(x.shape)-2

# reduce
tst = SubNetConv(3, padding=1, stride=2, ndim=xdim, norm_type=NormType.Batch, dropout=.1)(1,2,2)
y = tst(x)
test_eq(y.shape, [16,2,8,16,16])
print(tst)
# upsample
tst = SubNetConv(ks=4, padding=0, stride=4, ndim=xdim, norm_type=NormType.Batch, transpose=True)(2,1) # to double the size, the kernel cannot be odd
test_eq(tst(y).shape, [16,1,32,64,64])
print(tst)
del y
# ConvLayer(2*n_out_channels, n_out_channels, ks=ks, transpose=True, padding=(ks-1)//2)

Sequential(
  (0): Sequential(
    (0): ConvLayer(
      (0): Conv3d(1, 2, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), bias=False)
      (1): BatchNorm3d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (1): Dropout(p=0.1, inplace=False)
  )
  (1): Sequential(
    (0): ConvLayer(
      (0): Conv3d(2, 2, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), bias=False)
      (1): BatchNorm3d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (1): Dropout(p=0.1, inplace=False)
  )
)
ConvLayer(
  (0): ConvTranspose3d(2, 1, kernel_size=(4, 4, 4), stride=(4, 4, 4), bias=False)
  (1): BatchNorm3d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
)


In [ ]:
#| export
class _Net_recurse(nn.Module):
    def __init__(self, parameters):
        """Class for recursive definition of U-network.p

        Parameters:
        in_channels - (int) number of channels for input.
        mult_chan - (int) factor to determine number of output channels
        depth - (int) if 0, this subnet will only be convolutions that double the channel count.
        """
        super().__init__()
        # Parameters 
        self.depth = parameters.depth
        n_out = parameters.in_channels*parameters.mult_chan
        ks = parameters.kernel_size
        ndim = parameters.ndim
        n_in, n_conv = (parameters.in_channels, parameters.n_conv_per_depth)

        # Layers
        Pooling = parameters.pool(ks=parameters.pool_size, ndim=ndim)
        UpSample = nn.Upsample(scale_factor=parameters.pool_size, mode='nearest')
        SubNet_Conv = SubNetConv(ks=ks, stride=1,padding=None, bias=None, ndim=ndim, norm_type=parameters.norm_type, 
                                 bn_1st=True, act_cls=parameters.activation, transpose=False, dropout=parameters.dropout)
        
        # Blocks        
        self.sub_conv_more = SubNet_Conv(n_in, n_out, n_conv)        
        if self.depth > 0:
            parameters.in_channels = n_out; parameters.mult_chan = 2; parameters.depth=(self.depth - 1)
            self.sub_u = nn.Sequential(Pooling,                         # layer reducing the image size (usually a pooling layer)
                                       _Net_recurse(parameters),        # lower unet level
                                       UpSample,                        # layer increasing the image size (usually an upsampling layer)
                                       )
            self.sub_conv_less = SubNet_Conv(3*n_out, n_out, n_conv)

    def forward(self, x):
        if self.depth == 0:
            return self.sub_conv_more(x)
        else:  # depth > 0
            x_conv_more = self.sub_conv_more(x)                 # convolutions with increasing number of channels
            x_from_sub_u = self.sub_u(x_conv_more)
            x_cat = torch_cat((x_from_sub_u,x_conv_more), 1)    # concatenate the upsampled outputs of the lower level with the outputs of the next level in size
            x_conv_less = self.sub_conv_less(x_cat)             # convolutions with decreasing number of channels
        return x_conv_less

In [ ]:
#| export 
class UNet(nn.Module):
	def __init__(self,
				depth=4,
				mult_chan=32,
				in_channels=1,
				out_channels=1,
				last_activation=None,
				kernel_size=3,
				ndim=2,
				n_conv_per_depth=2,
				activation=nn.ReLU,
				norm_type=NormType.Batch,
				dropout=0.0,
				pool=MaxPool,
				pool_size=2,
				residual=False,
				prob_out=False,
				eps_scale=1e-3,
				):
		super().__init__()
		attributesFromDict(locals())

		parameters = self
		self.net_recurse = _Net_recurse(parameters)
		
		if last_activation is None:
			last_activation = activation if not residual else None
			self.last_activation = activation
		self.conv_out = ConvLayer(mult_chan, out_channels, ndim=ndim, ks=kernel_size, norm_type=None, act_cls=last_activation, padding=1)

	def forward(self, x):
		x_rec = self.net_recurse(x)
		final = self.conv_out(x_rec)

		if self.residual:
			if not (self.out_channels == self.in_channels): raise ValueError("number of input and output channels must be the same for a residual net.")
			final = final + x
		final = self.activation(final)

		if self.prob_out:
			scale = ConvLayer(self.out_channels, self.out_channels, ndim=self.ndim, ks=1, norm_type=None, act_cls=nn.Softplus)(x_rec)
			scale = Lambda(lambda x: x+np.float32(self.eps_scale))(scale)
			final = torch_cat((final,scale), 1)

		return final



In [ ]:
x = torch_randn(16,1,32,64,64)
xdim = len(x.shape)-2

tst = UNet(depth=1,ndim=xdim, n_conv_per_depth=1)
mods = list(tst.children())
print(mods)

[_Net_recurse(
  (sub_conv_more): ConvLayer(
    (0): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
    (1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (sub_u): Sequential(
    (0): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (1): _Net_recurse(
      (sub_conv_more): ConvLayer(
        (0): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
    )
    (2): Upsample(scale_factor=2.0, mode=nearest)
  )
  (sub_conv_less): ConvLayer(
    (0): Conv3d(96, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
    (1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
), ConvLayer(
  (0): Conv3d(32, 1, kernel_size=(3, 3, 3), stride=(1, 1,

Alternatively, we can use fastai unet builder


In [ ]:
from fastai.vision.all import unet_learner

In [ ]:
# import torch
# import torch.nn as nn
# from fastai.vision.learner import create_body
# from fastai.vision.models.unet import DynamicUnet
# from fastai.vision.all import *

# class RecursiveUNet(nn.Module):
#     def __init__(self, depth, num_classes):
#         super(RecursiveUNet, self).__init__()
#         self.depth = depth
#         self.num_classes = num_classes
#         self.encoder = create_body(models.resnet34, pretrained=True, cut=-2)
#         self.decoder = nn.ModuleList()
        
#         for i in range(self.depth):
#             if i == 0:
#                 self.decoder.append(DynamicUnet(self.encoder, self.num_classes))
#             else:
#                 self.decoder.append(DynamicUnet(self.decoder[i-1], self.num_classes))
        
#     def forward(self, x):
#         outputs = []
#         for i in range(self.depth):
#             if i == 0:
#                 outputs.append(self.decoder[i](x))
#             else:
#                 outputs.append(self.decoder[i](outputs[i-1][0]))
#         return outputs[-1]


In [ ]:
# learn = unet_learner(dls, models.resnet18, loss_func=F.l1_loss, n_in=1, n_out=1, pretrained=False, cut=None)
# learn.summary()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()